# TP 02 - Le Mécanisme d'Attention - CORRECTION

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Comprendre et implémenter le mécanisme d'attention

---

**VERSION ENSEIGNANT AVEC CORRECTIONS**

In [ ]:
!pip install torch matplotlib numpy -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math

torch.manual_seed(42)
print(f"PyTorch version: {torch.__version__}")

## Exercice 1 : Calcul des scores

In [ ]:
seq_len = 3
d_k = 4

Q = torch.randn(seq_len, d_k)
K = torch.randn(seq_len, d_k)
V = torch.randn(seq_len, d_k)

# ============================================
# CORRECTION EXERCICE 1
# ============================================
scores = Q @ K.T  # ou K.transpose(-2, -1) pour les batches

print("Scores (Q @ K^T):")
print(scores)
print(f"Shape: {scores.shape}")  # (3, 3)

## Exercice 2 : Scaling

In [ ]:
# ============================================
# CORRECTION EXERCICE 2
# ============================================
scaled_scores = scores / math.sqrt(d_k)

print("Scaled scores:")
print(scaled_scores)

## Exercice 3 : Softmax

In [ ]:
# ============================================
# CORRECTION EXERCICE 3
# ============================================
attention_weights = F.softmax(scaled_scores, dim=-1)

print("Poids d'attention (après softmax):")
print(attention_weights)
print(f"\nVérification - Somme par ligne: {attention_weights.sum(dim=-1)}")

## Exercice 4 : Output

In [ ]:
# ============================================
# CORRECTION EXERCICE 4
# ============================================
output = attention_weights @ V

print("Output:")
print(output)
print(f"Shape: {output.shape}")  # (3, 4)

## Exercice 5 : Fonction complète

In [ ]:
# ============================================
# CORRECTION EXERCICE 5
# ============================================
def scaled_dot_product_attention(Q, K, V):
    """
    Calcule le Scaled Dot-Product Attention.
    """
    d_k = K.shape[-1]
    
    # 1. Calculer les scores
    scores = Q @ K.transpose(-2, -1)
    
    # 2. Scaling
    scaled_scores = scores / math.sqrt(d_k)
    
    # 3. Softmax
    attention_weights = F.softmax(scaled_scores, dim=-1)
    
    # 4. Output
    output = attention_weights @ V
    
    return output, attention_weights

# Test
Q_test = torch.randn(4, 8)
K_test = torch.randn(4, 8)
V_test = torch.randn(4, 8)

output, weights = scaled_dot_product_attention(Q_test, K_test, V_test)

print(f"Structure de sortie : {output.shape}")  # (4, 8)
print(f"Structure des poids : {weights.shape}")  # (4, 4)
print(f"Somme des poids par ligne : {weights.sum(dim=-1)}")  # [1, 1, 1, 1]

## Exercice 6 : Classe SelfAttention

### Self-Attention vs Cross-Attention

**Self-Attention** (ce qu'on fait ici) :
- Q, K, V sont tous calculés à partir du **même** input `x`
- Chaque mot regarde les autres mots **de la même phrase**

**Cross-Attention** (on verra dans les projets) :
- Q vient d'une source, K et V d'une **autre** source
- Utilisé en traduction (décodeur interroge encodeur)

### D'où viennent Q, K, V ?

Dans les exercices 1-5 : tenseurs aléatoires pour comprendre le mécanisme.
En pratique : **Q, K, V sont calculés à partir des embeddings**.

| Rôle | Représentation | Question posée |
|------|----------------|----------------|
| **Query** | `Q = x @ W_q` | "Qu'est-ce que je cherche ?" |
| **Key** | `K = x @ W_k` | "Comment les autres me voient ?" |
| **Value** | `V = x @ W_v` | "Quelle info je transmets ?" |

Les matrices W_q, W_k, W_v sont **apprises** : le modèle découvre quelles "facettes" sont utiles pour chaque rôle.

In [ ]:
# ============================================
# CORRECTION EXERCICE 6
# ============================================
class SelfAttention(nn.Module):
    """
    Module de Self-Attention.
    
    Projette l'input x vers Q, K, V puis applique l'attention.
    """
    
    def __init__(self, embed_dim):
        super().__init__()
        self.embed_dim = embed_dim
        
        # Projections linéaires (3 matrices de poids distinctes)
        self.W_q = nn.Linear(embed_dim, embed_dim)
        self.W_k = nn.Linear(embed_dim, embed_dim)
        self.W_v = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x):
        # Projeter x vers Q, K, V
        Q = self.W_q(x)
        K = self.W_k(x)
        V = self.W_v(x)
        
        # Réutiliser la fonction de l'exercice 5
        return scaled_dot_product_attention(Q, K, V)

# Test
embed_dim = 32
batch_size = 2
seq_len = 5

attention_layer = SelfAttention(embed_dim)
x = torch.randn(batch_size, seq_len, embed_dim)

output, weights = attention_layer(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")  # (2, 5, 32)
print(f"Weights shape: {weights.shape}")  # (2, 5, 5)

## Section 7 : Visualiser l'attention d'un vrai modèle

Cette section utilise DistilBERT pour montrer des patterns d'attention réels.

### Tokens spéciaux : [CLS] et [SEP]

| Token | Signification | Rôle |
|-------|---------------|------|
| **[CLS]** | "Classification" | Ajouté au début. Son vecteur final représente toute la phrase. |
| **[SEP]** | "Separator" | Ajouté à la fin. Sépare les phrases. |

**Pourquoi [CLS] reçoit beaucoup d'attention ?** Il est entraîné pour "résumer" la phrase → normal qu'il attire l'attention.

### Aperçu : Multi-Head Attention

DistilBERT a **12 têtes par couche** × 6 couches = 72 patterns différents. Chaque tête capture des relations différentes (syntaxe, coréférence, positions...).

> **On verra le Multi-Head en TP 03.** Ici, on visualise une tête qui capture bien "it" → "cat".

In [ ]:
# Installation de transformers
!pip install transformers -q

from transformers import AutoModel, AutoTokenizer

# Charger DistilBERT
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_attentions=True)
model.eval()

# Phrase de test
phrase = "The cat sat on the mat because it was tired"
inputs = tokenizer(phrase, return_tensors="pt")
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)

attentions = outputs.attentions

print(f"Phrase: {phrase}")
print(f"Tokens: {tokens}")

# Visualisation - Couche 5, Tête 2 : capture bien "it" → "cat"
layer = 4   # Couche 5 (0-indexed)
head = 1    # Tête 2 (0-indexed)
attention_matrix = attentions[layer][0, head].numpy()

plt.figure(figsize=(10, 8))
plt.imshow(attention_matrix, cmap='Blues')
plt.xticks(range(len(tokens)), tokens, rotation=45, ha='right')
plt.yticks(range(len(tokens)), tokens)
plt.title(f"Attention réelle - Couche {layer+1}, Tête {head+1}")
plt.colorbar(label="Poids d'attention")
plt.tight_layout()
plt.show()

# Analyse du pronom "it"
it_index = tokens.index("it")
print(f"\nAttention de 'it' (Couche {layer+1}, Tête {head+1}) :")
print("-" * 40)
for token, weight in zip(tokens, attention_matrix[it_index]):
    bar = "█" * int(weight * 30)
    highlight = " ← antécédent !" if token == "cat" else ""
    print(f"  {token:10} {weight:.2f} {bar}{highlight}")

## Section 9 : Pour aller plus loin - GPT vs BERT

### Masque causal (GPT)

Le masque permet d'entraîner efficacement en un seul forward pass.

In [ ]:
# Implémentation du masque causal
def scaled_dot_product_attention_with_mask(Q, K, V, mask=None):
    """Attention avec masking optionnel."""
    d_k = K.shape[-1]
    scores = Q @ K.transpose(-2, -1) / math.sqrt(d_k)
    
    if mask is not None:
        scores = scores.masked_fill(mask, float('-inf'))
    
    attention_weights = F.softmax(scores, dim=-1)
    output = attention_weights @ V
    return output, attention_weights

# Créer un masque causal (triangulaire supérieur)
seq_len = 5
causal_mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()

print("Masque causal (True = masqué):")
print(causal_mask.int())

# Test
Q = torch.randn(seq_len, 8)
K = torch.randn(seq_len, 8)
V = torch.randn(seq_len, 8)

output, weights = scaled_dot_product_attention_with_mask(Q, K, V, causal_mask)

print("\nPoids d'attention avec masque:")
print(weights.round(decimals=2))
print("\n→ Chaque position ne voit que les positions précédentes (et elle-même)")